In [2]:
from scrape_courses import get_courses_from_dgscene, get_readable_course_name
from scrape_events import get_all_sanctioned_events
from scrape_ratings import get_round_ratings_for_tournament
from logger import logger
import json

In [ ]:
# TODO
# generate list of all courses
# write to json
courses = get_courses_from_dgscene()
course_names = {
    course: get_readable_course_name(course) for course in courses
}
with open('data/course_names.json', 'w') as f:
    json.dump(course_names, f, indent=4)

In [ ]:
with open('data/course_names.json') as f:
    course_names: dict = json.load(f)
course_events = {}
for i, course in enumerate(course_names):
    logger.info(f'Fetching event {i}/{len(course_names)}')
    course_events[course] = get_all_sanctioned_events(course)
with open('data/course_events.json', 'w') as f:
    json.dump(course_events, f, indent=4)

In [3]:
try:
    logger.info('Fetching ratings...')
    with open('data/course_events.json') as f:
        course_events: dict = json.load(f)
    with open('data/course_ratings.json') as f:
        course_ratings: dict = json.load(f)
    for course in course_events:
        if course in course_ratings:
            logger.info(f'Skipping {course} (already scraped)...')
            continue
        logger.info(f'Fetching ratings for {course}...')
        events = course_events[course]
        course_ratings[course] = {
            event: get_round_ratings_for_tournament(event) for event in events
        }
except BaseException as e:
    logger.info(f'Error fetching ratings: {e}')

with open('data/course_ratings.json', 'w') as f:
    json.dump(course_ratings, f, indent=4)
logger.info('Ratings fetched and dumped to course_ratings.json.')
logger.info('')

[07/Oct/2024 22:20:42] INFO - Fetching ratings...
[07/Oct/2024 22:20:42] INFO - Skipping 22_Birdwalk (already scraped)...
[07/Oct/2024 22:20:42] INFO - Skipping 4_Wing_Campground (already scraped)...
[07/Oct/2024 22:20:42] INFO - Skipping Alexander_Hamilton_Park (already scraped)...
[07/Oct/2024 22:20:42] INFO - Skipping Anderson_Park_AB (already scraped)...
[07/Oct/2024 22:20:42] INFO - Skipping Aperture_Park (already scraped)...
[07/Oct/2024 22:20:42] INFO - Skipping Aspen_Meadows (already scraped)...
[07/Oct/2024 22:20:42] INFO - Skipping Badlands_DGC (already scraped)...
[07/Oct/2024 22:20:42] INFO - Skipping Baileys_Crossing_Disc_Golf_Course (already scraped)...
[07/Oct/2024 22:20:42] INFO - Skipping Baker_Park_AB (already scraped)...
[07/Oct/2024 22:20:42] INFO - Skipping Barrett_Park (already scraped)...
[07/Oct/2024 22:20:42] INFO - Skipping Bear_Lake_Open (already scraped)...
[07/Oct/2024 22:20:42] INFO - Skipping bear-paw-par-3-12330 (already scraped)...
[07/Oct/2024 22:20:42